In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
log_address = './log/depth/M0_all_MAPE/'

In [ ]:
log_file = pd.read_pickle(log_address + 'visual_log.pkl')#depth_car_v1   baseline  baseline_normalized_input
log_file.columns

In [ ]:
feature_list = log_file['feature_list']
label_list = log_file['label_list']
end_of_epoch_results = [log_file[index] for index in log_file.columns[2:]]

In [ ]:
batch_size=3

In [ ]:
end_of_epoch_results[0][0][0].shape

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from utils import visualizer as vis

def f(num_slider, num_play, epoch_slider, epoch_play):
    num_slider -=1
    epoch_slider -=1
    plt.figure(figsize=(21,7))
    plt.subplot(1,3,1)
    plt.imshow(feature_list[0][0][num_slider])
    plt.subplot(1,3,2)
    plt.imshow(label_list[0][0][num_slider,:,:,0])
    plt.subplot(1,3,3)
    plt.imshow(end_of_epoch_results[epoch_slider][0][num_slider, :, :, 0])
#     plt.set_cmap('rainbow')
    plt.set_cmap('prism')

num_slider = widgets.IntSlider(description="Sample Num", min=1, max=batch_size, continuous_update=False)
num_play = widgets.Play(interval=1000,  min=1,    max=batch_size,    step=1,    description="Sample Num",    disabled=False)
widgets.jslink((num_slider, 'value'), (num_play, 'value'))

epoch_slider = widgets.IntSlider(description="Epoch Num", min=1, max=len(log_file.columns)-2, continuous_update=False)
epoch_play = widgets.Play(interval=1000,  min=1, max=len(log_file.columns),    step=1,    description="Epoch Num",    disabled=False)
widgets.jslink((epoch_slider, 'value'), (epoch_play, 'value'))

interact(f, 
         num_slider=num_slider,
         num_play=num_play,
         epoch_slider=epoch_slider,
         epoch_play=epoch_play)

In [ ]:
log_address = './log/depth/M2_all_MAPE/'

In [ ]:
metric_log = pd.read_pickle(log_address + 'metric_log.pkl')#depth_car_v1   baseline  baseline_normalized_input
metric_log.columns

In [ ]:
import plotly.express as px
fig = px.line(metric_log, x='epoch', y='val_MAPE') 
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(metric_log, x='epoch', y='val_complete_depth_map_MAPE') 
fig.show()

In [ ]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import plotly

In [ ]:
d = pd.DataFrame(columns=['x', 'y'])

In [ ]:
for i in range(100):
    d.loc[i] = [i, np.sin(i*0.1)]
    time.sleep(0.5)
    fig = px.line(d, x='x', y='y')
    plotly.io.write_html(fig, file="m.html")

# Evaluatation and Prediction

In [ ]:
ref_net_name = 'VIPER'

In [ ]:
# model.load_weights('./log/unet/' + ref_net_name + '/my_keras_model.h5')
model.load_weights(address +'my_keras_model.h5')

In [ ]:
steps = dsg_validation.size/dsg_validation.batch_size
result = model.evaluate_generator(dsg_validation, steps=steps, verbose=1)

In [ ]:
for i in result:
    print(i)

## Plot

In [ ]:
dsg_validation.index= 59
preview = iter(dsg_validation)
feature_list, label_list=next(preview)
model.evaluate(feature_list, label_list)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from utils import visualizer as vis

options=['Simple plot', 'RGB point cloud', 'Segmented point cloud']

def f(x):
    vis.draw_samples(feature_list, label_list, x-1,
                     feature_types=COMMON_SETTING['feature_types'], 
                     label_types=COMMON_SETTING['label_types'])
    
def f2(x):
    vis.draw_point_cloud(label_list[0][x-1], feature_list[0][x-1])
    
def view_type_selector(y):
    if y == options[0]:
        interact(f, 
                 x=widgets.IntSlider(description="Sample Num", min=1, max=dsg_train.batch_size, continuous_update=False))
    if y == options[1]:
        interact_manual(f2,
                        x=widgets.IntSlider(description="Sample Num", min=1, max=dsg_train.batch_size, continuous_update=False))
    else:
        print(y)
        
x=interact_manual(view_type_selector, y=widgets.Dropdown(description="View type:", options=options))

In [ ]:
i = 4
plt.figure(figsize=(20,20))
plt.set_cmap('hot')
plt.subplot(1,3,1)
plt.imshow(label_list[0][0, :, :, 0])
plt.subplot(1,3,2)
plt.imshow(predict[0, :, :, 0])
plt.subplot(1,3,3)
plt.imshow((label_list[0][0, :, :, 0]-predict[0, :, :, 0])/(label_list[0][0, :, :, 0]+0.0000001))


In [ ]:
(label_list[0][0, :, :, 0]-predict[0, :, :, 0]).sum()

In [ ]:
vis.draw_point_cloud(predict[0, :, :, 0], feature_list[0][0])

In [ ]:
vis.draw_point_cloud(predict[0, :, :, 0], feature_list[0][0])

In [ ]:
vis.draw_point_cloud(label_list[0][0, :, :, 0], feature_list[0][0])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

In [ ]:
image =  feature_list[0][0]
depth = predict[0, :, :, 0]*100

In [ ]:
a = o3d.geometry.Image(image)  #pylint: disable=no-member
b = o3d.geometry.Image(depth)  #pylint: disable=no-member
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(a, b)  #pylint: disable=no-member
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(  #pylint: disable=no-member
    rgbd_image,
    o3d.camera.PinholeCameraIntrinsic(  #pylint: disable=no-member
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))  #pylint: disable=no-member

# rgb_vector = np.reshape(image, [-1, 3])
# pcd.colors = o3d.utility.Vector3dVector(rgb_vector)  #pylint: disable=no-member
o3d.visualization.draw_geometries([pcd])  #pylint: disable=no-member

# Plotly

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
import plotly

In [ ]:
deptmap = label_list[0][0][:,:,0]

In [ ]:
x_l = []
y_l = []
z_l = []
c_l = []
s_l = []
for x in range(deptmap.shape[0]):
    for y in range(deptmap.shape[1]):
        x_l.append(x)
        y_l.append(y)
        z_l.append(deptmap[x,y]*1000)
        c_l.append(deptmap[x,y]*1000)
        s_l.append(0.1)
df = pd.DataFrame({'x':x_l, 'y':y_l, 'z':z_l,  'color':c_l, 'size':s_l})

In [ ]:
fig = px.scatter(df, x='y', y='x',color='color', size='size', hover_name='color',  render_mode='webgl')

In [ ]:
# iris = px.data.iris()
fig = px.scatter_3d(df, x='x', y='y', z='z', color='color', size='size')
# fig = px.scatter(df, x='y', y='x',color='color', size='size', hover_name='color',  render_mode='webgl')
fig.data[0].marker.line.width=0
fig.data[0].marker.size = 1
fig.layout.yaxis.autorange='reversed'
fig.show()

In [ ]:
plotly.io.write_html(fig, file="m.html")

# Test

In [ ]:
d = label_list[0][0][0,:,:,0]

In [ ]:
plt.imshow(d)

In [ ]:
x=d==1000

In [ ]:
d.size

In [ ]:
h = np.histogram(d, 10)

In [ ]:
plt.hist(d.flatten(), 10)

In [ ]:
h[0]

In [ ]:
h[1]

In [ ]:
plt.bar(h[1][1:], h[0])